# Getting Started

This notebook showcases basic functionality of the code base.

Here, we load the metadata, an example dataset, and run inference using a pre-trained model. 

We also show how to visualize the joint angle predictions using a hand mesh (requires the UmeTrack package -- see README.md).

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
DATA_DOWNLOAD_DIR = Path.home()

## Download Dataset Metadata

In [ ]:
!cd {DATA_DOWNLOAD_DIR} && curl https://fb-ctrl-oss.s3.amazonaws.com/emg2pose/emg2pose_metadata.csv -o emg2pose_metadata.csv

In [ ]:
import pandas as pd

metadata_df = pd.read_csv(DATA_DOWNLOAD_DIR / "emg2pose_metadata.csv")
metadata_df.head(5)

## Download a Smaller (~600 MiB) Version of the Dataset

In [ ]:
!cd {DATA_DOWNLOAD_DIR} && curl "https://fb-ctrl-oss.s3.amazonaws.com/emg2pose/emg2pose_dataset_mini.tar" -o emg2pose_dataset_mini.tar

# Unpack the tar to ~/emg2pose_dataset_mini
!tar -xvf emg2pose_dataset_mini.tar

In [ ]:
import glob
import os

sessions = sorted(glob.glob(os.path.join(DATA_DOWNLOAD_DIR, "emg2pose_dataset_mini/*.hdf5")))
sessions

## Let's Look at a Dataset

In [ ]:
from emg2pose.data import Emg2PoseSessionData

session = sessions[15]
data = Emg2PoseSessionData(hdf5_path=session)

In [ ]:
print(data.fields)
print()

print(f"{'emg shape: ':<20} {data['emg'].shape}")
print(f"{'joint_angles shape: ':<20} {data['joint_angles'].shape}")
print(f"{'time shape: ':<20} {data['time'].shape}")

In [ ]:
metadata_df[metadata_df["filename"] == data.metadata["filename"]]

In [ ]:
import emg2pose.visualization as visualization

visualization.ik_failure_plot(data)

In [ ]:
from emg2pose.utils import downsample
import numpy as np

joint_angles = data["joint_angles"]
joint_angles_30hz = downsample(joint_angles, native_fs=2000, target_fs=30)

assert not np.any(np.isnan(joint_angles_30hz))

visualization.plot_hand_mesh(joint_angles_30hz[100], auto_range=False)

In [ ]:
import numpy as np


import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

In [ ]:
visualization.get_plotly_animation_for_joint_angles(joint_angles_30hz[0:250])

### Render the Plotly Animation to Video Frames

In [ ]:
import mediapy

frames = visualization.joint_angles_to_frames_parallel(joint_angles_30hz[0:250])
frames = visualization.remove_alpha_channel(frames)
mediapy.show_video(frames, width=800, fps=30, downsample=True)

## Let's Load a Checkpoint and Generate some Predictions

In [ ]:
!cd {DATA_DOWNLOAD_DIR} \
&& curl "https://fb-ctrl-oss.s3.amazonaws.com/emg2pose/emg2pose_model_checkpoints.tar.gz" -o emg2pose_model_checkpoints.tar.gz && \
tar -xvzf emg2pose_model_checkpoints.tar.gz

In [ ]:
from emg2pose.utils import generate_hydra_config_from_overrides

config = generate_hydra_config_from_overrides(
    overrides=[
        "experiment=tracking_vemg2pose",
        f"checkpoint={DATA_DOWNLOAD_DIR}/emg2pose_model_checkpoints/regression_vemg2pose.ckpt"
    ]
)

In [ ]:
from emg2pose.lightning import Emg2PoseModule

module = Emg2PoseModule.load_from_checkpoint(
    config.checkpoint,
    network=config.network,
    optimizer=config.optimizer,
    lr_scheduler=config.lr_scheduler,
)

In [ ]:
session = data
start_idx = 0
stop_idx = 10_000

In [ ]:
import torch

session_window = session[start_idx:stop_idx]

# no_ik_failure is not a field so we slice separately
no_ik_failure_window = session.no_ik_failure[start_idx:stop_idx]

batch = {
    "emg": torch.Tensor([session_window["emg"].T]),  # BCT
    "joint_angles": torch.Tensor([session_window["joint_angles"].T]),  # BCT
    "no_ik_failure": torch.Tensor([no_ik_failure_window]),  # BT
}

preds, joint_angles, no_ik_failure = module.forward(batch)

# Algorithms that use the initial state for ground truth will do poorly
# when the first joint angles are missing!
if (joint_angles[:, 0] == 0).all():
    print(
        "Warning! Ground truth not available at first time step!"
    )

# BCT --> TC (as numpy)
preds = preds[0].T.detach().numpy()
joint_angles = joint_angles[0].T.detach().numpy()

In [ ]:
preds.shape

In [ ]:
joint_angles.shape

In [ ]:
joint_angles_30hz = downsample(joint_angles, native_fs = 2000, target_fs = 30)
visualization.get_plotly_animation_for_joint_angles(joint_angles_30hz[0:250], color="gray")

In [ ]:
preds_30hz = downsample(preds, native_fs=2000, target_fs=30)
visualization.get_plotly_animation_for_joint_angles(preds_30hz[0:250], color="lightpink")

### Compare the Ground Truth and Predictions Side-by-Side

In [ ]:
gt_frames = visualization.joint_angles_to_frames_parallel(joint_angles_30hz[0:250], color="gray")
pred_frames = visualization.joint_angles_to_frames_parallel(preds_30hz[0:250], color="lightpink")

gt_frames = visualization.remove_alpha_channel(gt_frames)
pred_frames = visualization.remove_alpha_channel(pred_frames)

In [ ]:
mediapy.show_videos(dict(gt=gt_frames, pred=pred_frames), width=400, fps=30, downsample=True)

In [ ]:
N_COLS = 2
N_ROWS = 10

fig, axs = plt.subplots(N_ROWS, N_COLS, figsize=(4*N_COLS, 2*N_ROWS))

axs_flattened = axs.flatten()
for i, ax in enumerate(axs_flattened):
    ax.set_title(f"Joint Angle {i}")
    ax.plot(joint_angles_30hz[:, i], label="gt")
    ax.plot(preds_30hz[:, i], label="pred")

    ax.legend()

fig.suptitle("Predicted vs. Ground Truth Joint Angles")

plt.tight_layout()
fig.subplots_adjust(top=0.95)

plt.show()